### Libraries Importation

In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

pd.set_option('display.max_columns', None)

### Data importation 

In [2]:
# Data for training
features=pd.read_csv('data/Train_transactions.csv')
target=pd.read_csv('data/Train_customers_repurchase.csv')

C:\Users\lucas\AppData\Local\Temp\ipykernel_10048\2046257610.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  features=pd.read_csv('data/Train_transactions.csv')


#### Copy 

In [3]:
features_copy = features.copy()
target_copy = target.copy()

## Feature Engenering 

In [4]:
print("There are ", len(features_copy) , " transactions")

There are  3298736  transactions


In [5]:
print(features_copy.columns)

Index(['id_client', 'transaction_date', 'stores_nb', 'item_count',
       'gross_amount', 'discount_amount', 'basket_value', 'payment_gift',
       'payment_cheque', 'payment_cash', 'payment_card', 'email_domain',
       'civility', 'zip_code', 'card_subscription', 'multicard',
       'price_segmentation'],
      dtype='object')


In [6]:
features_copy.head()

,id_client,transaction_date,stores_nb,item_count,gross_amount,discount_amount,basket_value,payment_gift,payment_cheque,payment_cash,payment_card,email_domain,civility,zip_code,card_subscription,multicard,price_segmentation
0,37520523,2019-12-26T16:44:00.000Z,12015,57,138.69,0.49,138.20,0,0,1,0,NaN,MME,12420,NaN,0.0,NaN
1,84917629,2019-08-01T12:16:00.000Z,77131,6,9.87,0.00,9.87,0,0,1,0,NaN,MME,77450,NaN,0.0,NaN
2,20962900,2019-09-25T18:10:00.000Z,92786,6,26.25,2.21,24.04,0,0,0,1,GMAIL.COM,MME,92110,NaN,0.0,Quali
3,85917543,2020-01-24T17:34:00.000Z,21554,5,15.08,0.00,15.08,0,0,0,1,SFR.FR,M,21130,NaN,0.0,Mixte
4,6861392,2019-08-22T19:28:00.000Z,83820,25,81.16,30.60,50.56,0,0,0,1,ORANGE.FR,MME,83600,NaN,0.0,Mixte


#### Count if duplicated

In [7]:
features_copy.duplicated().sum()

np.int64(0)

-   We don't have any duplicated row ! 

#### Count number NaN by cols 

In [8]:
features_copy.isna().mean()

id_client             0.000000
transaction_date      0.000000
stores_nb             0.000000
item_count            0.000000
gross_amount          0.000000
discount_amount       0.000000
basket_value          0.000000
payment_gift          0.000000
payment_cheque        0.000000
payment_cash          0.000000
payment_card          0.000000
email_domain          0.388485
civility              0.001976
zip_code              0.092311
card_subscription     0.934478
multicard             0.001976
price_segmentation    0.473834
dtype: float64

-   `email_domain` = 39% of **NaN**
-   `civility` < 1% of **NaN**
-   `zip_code` = 10% of **NaN**
-   `card_subscription` = 9% of **NaN**    
-   `multicard` < 1% of **NaN** 
-   `price_segmentation    ` = 47% of **NaN** 

Among all the features: we will not interested by: card_subscription  (The date of Subscription card but it concerned to less customers)

#### First: Clean some variables

In [9]:
def clean_zip(x):
    if pd.isna(x):
        return 0
    x = str(x) #be sure we have string
    if x.isdigit() and len(x) >=2:
        return int(x[:2])   # return 77, 75, etc.
    
    # case where it's not digit (ex BE for belgium)
    return 0

### create binary feature if email is provided
def clean_mail(x):
    if pd.isna(x):
        return 0
    else : 
        return 1
    

In [10]:
# get binary variable: 1 female and 0 for male
features_copy["sex"] = features_copy["civility"].map ({'M':0, 'MME':1, 'NR':np.nan})
#clean transaction (use format  YYYY-MM-DD)
features_copy["transaction_date_clean"] = pd.to_datetime(features_copy["transaction_date"].str[:10])
#clean zip code (keep only first two digits)
features_copy["zip_code_clean"]=features_copy["zip_code"].apply(clean_zip)
# create binary feature if email is provided
features_copy["is_mail"] = features_copy["email_domain"].apply(clean_mail)
# create binary features for multicard and price_segmentation
features_copy["multicard_clean"]= features_copy["multicard"].apply(lambda x: 1 if x else 0)
features_copy["is_access"] = features_copy["price_segmentation"].apply(lambda x: 1 if x == "Access" else 0)
features_copy["is_mixte"] = features_copy["price_segmentation"].apply(lambda x: 1 if x == "Mixte" else 0)
features_copy["is_quali"] = features_copy["price_segmentation"].apply(lambda x: 1 if x == "Quali" else 0)

In [11]:
# create new features for target cities customers (it will not be used for prediction)
# Zip of top 4 cities in France
Paris_zip = [75, 77, 78, 91, 92, 93, 94, 95]  
Marseille_zip = [13]
Lyon_zip = [69]
Toulouse_zip = [31]

# Create binary features for each city
features_copy["is_paris"] = features_copy["zip_code_clean"].apply(lambda x: 1 if x in Paris_zip else 0)
features_copy["is_marseille"] = features_copy["zip_code_clean"].apply(lambda x: 1 if x in Marseille_zip else 0)
features_copy["is_lyon"] = features_copy["zip_code_clean"].apply(lambda x: 1 if x in Lyon_zip else 0)
features_copy["is_toulouse"] = features_copy["zip_code_clean"].apply(lambda x: 1 if x in Toulouse_zip else 0)


In [12]:

features_merge = features_copy.merge(target_copy, on="id_client", how="left")
pd.crosstab(features_merge["is_quali"], features_merge["repurchase"], normalize='index')

repurchase,0,1
is_quali,,
0,0.758320,0.241680
1,0.765403,0.234597


In [13]:
features_copy.columns

Index(['id_client', 'transaction_date', 'stores_nb', 'item_count',
       'gross_amount', 'discount_amount', 'basket_value', 'payment_gift',
       'payment_cheque', 'payment_cash', 'payment_card', 'email_domain',
       'civility', 'zip_code', 'card_subscription', 'multicard',
       'price_segmentation', 'sex', 'transaction_date_clean', 'zip_code_clean',
       'is_mail', 'multicard_clean', 'is_access', 'is_mixte', 'is_quali',
       'is_paris', 'is_marseille', 'is_lyon', 'is_toulouse'],
      dtype='object')

In [14]:
#drop columns: payment_cheque, payment_cash,payment_card, card_subscription, email_domain, civility, zip_code, multicard, transaction_date) 
features_drop = features_copy.drop(columns=["payment_cheque", "payment_cash","payment_card", "card_subscription", "email_domain", "civility", "zip_code", "multicard", "transaction_date","price_segmentation","email_domain"])
features_drop.head()

,id_client,stores_nb,item_count,gross_amount,discount_amount,basket_value,payment_gift,sex,transaction_date_clean,zip_code_clean,is_mail,multicard_clean,is_access,is_mixte,is_quali,is_paris,is_marseille,is_lyon,is_toulouse
0,37520523,12015,57,138.69,0.49,138.20,0,1.0,2019-12-26,12,0,0,0,0,0,0,0,0,0
1,84917629,77131,6,9.87,0.00,9.87,0,1.0,2019-08-01,77,0,0,0,0,0,1,0,0,0
2,20962900,92786,6,26.25,2.21,24.04,0,1.0,2019-09-25,92,1,0,0,0,1,1,0,0,0
3,85917543,21554,5,15.08,0.00,15.08,0,0.0,2020-01-24,21,1,0,0,1,0,0,0,0,0
4,6861392,83820,25,81.16,30.60,50.56,0,1.0,2019-08-22,83,1,0,0,1,0,0,0,0,0


### Col for diff between two purchase

In [ ]:
features_drop = features_drop.sort_values(by=['id_client', 'transaction_date_clean'])
features_drop['diff'] = features_drop.groupby('id_client')['transaction_date_clean'].diff().dt.days
features_drop['diff_mean'] = features_drop.groupby('id_client')['diff'].transform('mean')

### Have the max and min date of the data

In [15]:
max_date = features_drop["transaction_date_clean"].max()
min_date = features_drop["transaction_date_clean"].min()
print("Max date:", max_date)
print("Min date:", min_date)

Max date: 2020-07-31 00:00:00
Min date: 2019-08-01 00:00:00


### create new variable by month 

In [ ]:
features_drop["month"] = pd.to_datetime(features_drop["transaction_date_clean"]).dt.month
features_drop["year"] = pd.to_datetime(features_drop["transaction_date_clean"]).dt.year
features_drop['month_year'] = features_drop['year'].astype(str) + '-' + features_drop['month'].astype(str)
features_drop['month_year'] = features_drop['year'].astype(str) + '-' + features_drop['month'].astype(str)

### data engering by months 

In [45]:
features_drop["nb_month"] = features_drop.groupby(["id_client"])["month_year"].transform("nunique")
features_drop["freq_month"] = features_drop.groupby(["id_client","month_year"])["month_year"].transform("count")
features_drop[features_drop["id_client"]==291][["transaction_date_clean","month_year","nb_month","freq_month"]].head(3)

,transaction_date_clean,month_year,nb_month,freq_month
826510,2019-08-02,2019-8,12,6
803986,2019-08-09,2019-8,12,6
1889835,2019-08-12,2019-8,12,6


In [42]:
unique_monthly = features_drop[['id_client', 'month_year', 'freq_month']].drop_duplicates()

#### last month number visit

In [ ]:
# Get frequency for the last month in the dataset
unique_monthly['freq_last_month'] = unique_monthly[unique_monthly['month_year'] == max_date]['freq_month']
# Replace NaN value by 0
unique_monthly['freq_last_month'] = unique_monthly['freq_last_month'].fillna(0)
# Compute mean freq by month and freq for last month
freq_mean_month = (
    unique_monthly.groupby('id_client')
    .agg(
        freq_mean_month=('freq_month', 'mean'),
        freq_last_month=('freq_last_month', 'sum')
    )
)
print(freq_mean_month.head())

,id_client,month_year,freq_month,freq_last_month
826510,291,2019-8,6,0.0
31026,291,2019-9,6,0.0
1152014,291,2019-10,5,0.0
378386,291,2019-11,5,0.0
1153979,291,2019-12,4,0.0


### Let's group by 

In [56]:
agg_features=(
    features_drop
    .groupby("id_client")
    .agg(
        frequency=("stores_nb", "count"),
        total_amount=("basket_value", "sum"),
        average_amount=("basket_value", "mean"),
        payment_gift=("payment_gift", "sum"),
        sex = ("sex", "max"),
        discount_amount=("discount_amount", "mean"),
        card_subscription=("multicard_clean", "max"),
        last_purchase=("transaction_date_clean", "max"),
        nb_unique_store=("stores_nb", "nunique"),
        nb_item=("item_count", "sum"),
        #repurchase=("repurchase", "max"),
        diff_mean=("diff_mean", "max"),
        nb_month=("nb_month", "max")
)
.reset_index()
)

In [57]:
agg_features.head()

,id_client,frequency,total_amount,average_amount,payment_gift,sex,discount_amount,card_subscription,last_purchase,nb_unique_store,nb_item,diff_mean,nb_month
0,291,55,4031.44,73.298909,5,1.0,2.181636,0,2020-07-31,1,1499,6.740741,12
1,365,25,388.08,15.523200,1,0.0,0.078800,0,2020-06-09,1,140,11.083333,9
2,1800,78,7021.10,90.014103,21,1.0,11.272179,0,2020-07-25,4,1961,4.662338,12
3,3471,60,1195.23,19.920500,0,1.0,0.142167,0,2020-03-14,2,406,3.644068,8
4,3961,3,413.72,137.906667,0,1.0,0.540000,0,2019-09-25,1,131,3.500000,1


In [ ]:

features_drop

,id_client,stores_nb,item_count,gross_amount,discount_amount,basket_value,payment_gift,sex,transaction_date_clean,zip_code_clean,is_mail,multicard_clean,is_access,is_mixte,is_quali,is_paris,is_marseille,is_lyon,is_toulouse,month,year,month_year,nb_month_visited,total_visit,diff,diff_mean,nb_month,freq_month
826510,291,92686,17,47.22,0.44,46.78,0,1.0,2019-08-02,92,0,0,0,0,1,1,0,0,0,8,2019,2019-8,12,55,NaN,6.740741,12,6
803986,291,92686,21,62.77,0.00,62.77,0,1.0,2019-08-09,92,0,0,0,0,1,1,0,0,0,8,2019,2019-8,12,55,7.0,6.740741,12,6
1889835,291,92686,15,50.94,0.00,50.94,0,1.0,2019-08-12,92,0,0,0,0,1,1,0,0,0,8,2019,2019-8,12,55,3.0,6.740741,12,6
3075340,291,92686,16,44.78,0.00,44.78,0,1.0,2019-08-14,92,0,0,0,0,1,1,0,0,0,8,2019,2019-8,12,55,2.0,6.740741,12,6
1954961,291,92686,20,74.08,0.77,73.31,0,1.0,2019-08-23,92,0,0,0,0,1,1,0,0,0,8,2019,2019-8,12,55,9.0,6.740741,12,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2965193,88010370,14008,6,37.83,0.00,37.83,0,1.0,2020-07-06,14,0,0,1,0,0,0,0,0,0,7,2020,2020-7,10,24,10.0,14.608696,10,2
1354706,88010370,14008,31,72.45,1.30,71.15,0,1.0,2020-07-14,14,0,0,1,0,0,0,0,0,0,7,2020,2020-7,10,24,8.0,14.608696,10,2
565440,88010403,69918,5,8.12,0.78,7.34,0,1.0,2019-09-16,69,1,0,0,1,0,0,0,1,0,9,2019,2019-9,3,3,NaN,44.500000,3,1
1484213,88010403,69918,4,7.63,0.00,7.63,0,1.0,2019-10-17,69,1,0,0,1,0,0,0,1,0,10,2019,2019-10,3,3,31.0,44.500000,3,1
